### H-Pile Design 
#### Per NHI Courses No. 132021 and 132022 "Design and Construction of Driven Pile Foundations - Volume 1 & 2"
####     "LRFD Bridge Design Specifications" 9th Edition 2020 

In [1]:
import dash
from dash import dcc, html, Input, Output, State
from dash.dependencies import Input, Output
import pandas as pd
from dash.exceptions import PreventUpdate
import base64
import io
import json

#### Pile Structural Information

In [ ]:
# Load CSV with all the H-Pile info
H_Pile = pd.read_csv("https://raw.githubusercontent.com/geotechnick/Geotechnical_Design/main/Pile_Design/H_Piles/HP_Spec_Table.csv")
Fy = 50  # Pile yield stress in ksi
pile_names = H_Pile['Section_E'].unique().tolist()

# Filter out the metric values
H_Pile = H_Pile.loc[:, ~H_Pile.columns.str.endswith('_M')]

# Create drop down with the pile names
dropdown = dcc.Dropdown(id='pile-dropdown', options=[{'label': name, 'value': name} for name in pile_names], value=pile_names[0])

# Define app layout
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("H-Pile Info"),
    dropdown,
    html.Div(id='pile-info')
])

# Define callback to update the displayed row
@app.callback(
    Output('pile-info', 'children'),
    [Input('pile-dropdown', 'value')]
)
def display_row(selected_value):
    if selected_value is None:
        return html.Div("No pile selected")
    
    pile_info = H_Pile[H_Pile['Section_E'] == selected_value]  # Creates dataframe that only has the selected value in it
    pile_units = H_Pile[H_Pile['Section_E'] == 'Units_English']  # Create separate dataframe that has units
    pile_display = pd.concat([pile_info, pile_units], ignore_index=True)  # Combine pile properties and pile units into single dataframe
    
    # Adds case_1 to the data frame
    # Get the current index label of the first row
    old_index_label = pile_info.index[0]

    # Define the new label for the first row
    new_index_label = 'case_1'

    # Rename the first row
    pile_info = pile_info.rename(index={old_index_label: new_index_label})

    # Now the first row is renamed and shifted
    pile_info.reset_index(inplace=True)

    return html.Table([html.Tr([html.Th(col), html.Td(pile_display[col].values[0])]) for col in pile_display.columns])

if __name__ == '__main__':
    app.run_server(debug=True)

#### Upload Soil Profile

In [ ]:
# Define app layout
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("File Upload"),
    dcc.Upload(
        id='upload-data',
        children=html.Div([
            'Drag and Drop or ',
            html.A('Select Files')
        ]),
        style={
            'width': '100%',
            'height': '60px',
            'lineHeight': '60px',
            'borderWidth': '1px',
            'borderStyle': 'dashed',
            'borderRadius': '5px',
            'textAlign': 'center',
            'margin': '10px'
        },
        # Allow multiple files to be uploaded
        multiple=True
    ),
    html.Div(id='output-data-upload')
])

# Function to handle file upload
def parse_contents(contents, filename):
    content_type, content_string = contents.split(',')

    decoded = base64.b64decode(content_string)
    
    try:
        if 'csv' in filename:
            # Assume the uploaded file is a CSV file
            df = pd.read_csv(io.StringIO(decoded.decode('utf-8')))
        elif 'xls' in filename:
            # Assume the uploaded file is a Excel file
            df = pd.read_excel(io.BytesIO(decoded))
    except Exception as e:
        print(e)
        return html.Div([
            'There was an error processing this file.'
        ])

    return html.Div([
        html.H5(filename),
        html.Hr(),  # horizontal line
        # Display the first 10 rows of the uploaded file
        html.Div(df.head(10))
    ])

# Callback to update output based on file upload
@app.callback(Output('output-data-upload', 'children'),
              [Input('upload-data', 'contents')],
              [State('upload-data', 'filename')])
def update_output(list_of_contents, list_of_names):
    if list_of_contents is None or list_of_names is None:
        raise PreventUpdate
    
    children = [
        parse_contents(c, n) for c, n in
        zip(list_of_contents, list_of_names)]
    return children

if __name__ == '__main__':
    app.run_server(debug=True)

In [10]:
#Should add Pile_Axial_Info with ground_surface, top_pile, pile_length, predrill_depth, steel_strength
# Define the widgets for user input
ground_surface_text = widgets.Text(description="Ground Surface:")
top_pile_text = widgets.Text(description="Top Pile:")
pile_length_float = widgets.FloatText(description="Pile Length:")
predrill_depth_float = widgets.FloatText(description="Predrill Depth:")
steel_strength_float = widgets.FloatText(description="Steel Strength:")

# Function to handle user input and create DataFrame
def create_dataframe(ground_surface, top_pile, pile_length, predrill_depth, steel_strength):
    global Pile_Axial_Info
    data = {
        "Ground Surface": [ground_surface],
        "Top Pile": [top_pile],
        "Pile Length": [pile_length],
        "Predrill Depth": [predrill_depth],
        "Steel Strength": [steel_strength]
    }
    Pile_Axial_Info = pd.DataFrame(data)
    
    ## adds case_1 to the data frame
    # Get the current index label of the first row
    old_index_label = Pile_Axial_Info.index[0]

    # Define the new label for the first row
    new_index_label = 'case_1'

    # Rename the first row
    Pile_Axial_Info = Pile_Axial_Info.rename(index={old_index_label: new_index_label})

    # Now the first row is renamed and shifted
    Pile_Axial_Info.reset_index(inplace=True)

    return Pile_Axial_Info

# Button to trigger dataframe creation
submit_button = widgets.Button(description="Submit")

def on_submit_button_clicked(b):
    display(create_dataframe(ground_surface_text.value, top_pile_text.value, pile_length_float.value, predrill_depth_float.value, steel_strength_float.value))

submit_button.on_click(on_submit_button_clicked)

# Display the widgets and button
input_widgets = widgets.VBox([ground_surface_text, top_pile_text, pile_length_float, predrill_depth_float, steel_strength_float, submit_button])
display(input_widgets)

,index,Ground Surface,Top Pile,Pile Length,Predrill Depth,Steel Strength
0,case_1,0,1,100.0,5.0,50.0


### Save properties to JSON

In [4]:

app = dash.Dash(__name__)

def save_json_to_file(json_data, file_path):
    if not file_path.endswith('.json'):
        file_path += '.json'
    with open(file_path, 'w') as json_file:
        json.dump(json_data, json_file, indent=4)
    return f"JSON data saved to {file_path}"

def save_json_widget(json_data):
    file_input = dcc.Input(id='file-path', type='text', placeholder='Enter file path', style={'width': '80%'})
    save_button = html.Button('Save JSON', id='save-button', n_clicks=0)
    output_div = html.Div(id='output')

    @app.callback(
        Output('output', 'children'),
        [Input('save-button', 'n_clicks')],
        [State('file-path', 'value')]
    )
    def save_json(n_clicks, file_path):
        if n_clicks > 0:
            if file_path:
                try:
                    json_data = json.loads(json_data)  # Load json_data again, in case it's modified elsewhere
                    return save_json_to_file(json_data, file_path)
                except Exception as e:
                    return f"Error: {str(e)}"
            else:
                return "Please enter a valid file path."

    return html.Div([file_input, save_button, output_div])

# Create JSON File ###############################
# Convert DataFrames to dictionaries
dict1 = Pile_Info.to_dict(orient='records')
dict2 = Layer_Properties.to_dict(orient='records')
dict3 = Pile_Axial_Info.to_dict(orient='records')

# Create a dictionary with dictionaries
pile_dict = {'Pile_Info': dict1, 'Layer_Properties': dict2, 'Pile_Axial_Info': dict3}

# Convert the dictionary to JSON format
json_data = json.dumps(pile_dict, indent=4)
###################################################

app.layout = save_json_widget(json_data)

if __name__ == '__main__':
    app.run_server(debug=True)

NameError: name 'Pile_Info' is not defined

### Upload from save file


In [12]:
def handle_upload(change):
    global Pile_Info
    global Layer_Properties
    global Pile_Axial_Info
    uploaded_file = change['new'][0]
    content = uploaded_file['content']
    content_bytes = content.tobytes()
    content_str = content_bytes.decode('utf-8')
    data = json.loads(content_str)

    # Check if 'Pile_Info' key exists and create a DataFrame
    if 'Pile_Info' in data:
        Pile_Info = pd.DataFrame(data['Pile_Info'])
        print("DataFrame for 'Pile_Info':")
        print(Pile_Info)
        # Optionally, you can save the DataFrame to a file
        # pile_info_df.to_csv("pile_info_data.csv", index=False)
    else:
        print("The key 'Pile_Info' does not exist in the JSON data.")

    # Check if 'Soil_Properties' key exists and create a DataFrame
    if 'Layer_Properties' in data:
        Layer_Properties = pd.DataFrame(data['Layer_Properties'])
        print("\nDataFrame for 'Layer_Properties':")
        print(Layer_Properties)
        # Optionally, you can save the DataFrame to a file
        # soil_properties_df.to_csv("soil_properties_data.csv", index=False)
    else:
        print("The key 'Layer_Properties' does not exist in the JSON data.")

        # Check if 'Soil_Properties' key exists and create a DataFrame
    if 'Pile_Axial_Info' in data:
        Pile_Axial_Info = pd.DataFrame(data['Pile_Axial_Info'])
        print("\nDataFrame for 'Pile_Axial_Info':")
        print(Pile_Axial_Info)
        # Optionally, you can save the DataFrame to a file
        # soil_properties_df.to_csv("soil_properties_data.csv", index=False)
    else:
        print("The key 'Pile_Axial_Info' does not exist in the JSON data.")

# Creating file upload widget
file_upload = widgets.FileUpload(accept='.json')

# Attaching the event handler
file_upload.observe(handle_upload, names='value')

# Displaying the widget
display(file_upload)

FileUpload(value=(), accept='.json', description='Upload')

In [122]:
class pile_design:
    
    def __init__(self):
        self.Pile_Info = Pile_Info
        self.Layer_Properties = Layer_Properties
        self.Pile_Axial_Info = Pile_Axial_Info
    
    def Pile_Effective_Stress (self):
        self.top_pile = pd.to_numeric(self.Pile_Axial_Info.iloc[0]['Top Pile'])
        self.gs = pd.to_numeric(self.Pile_Axial_Info.iloc[0]['Ground Surface'])
        self.pile_length = pd.to_numeric(self.Pile_Axial_Info.iloc[0]['Pile Length'])
        self.bot_pile = pd.to_numeric(self.Pile_Axial_Info.iloc[0]['Top Pile']) - pd.to_numeric(self.Pile_Axial_Info.iloc[0]['Pile Length'])
        self.predrill = pd.to_numeric(self.Pile_Axial_Info.iloc[0]['Predrill Depth'])
        self.water_level = pd.to_numeric(self.Layer_Properties.iloc[0]['mlv_top_elev'])
        
        self.pile_ruler = []
        for i in range(int(self.pile_length) + 1):
            self.pile_ruler.append(self.top_pile - i)

        self.eff_stress = []
        loop_count = 0
        loop_list = []
        for i in self.pile_ruler:
            if i > self.gs:
                self.eff_stress.append(0)
            
            elif i >= (self.gs - self.predrill):
                self.eff_stress.append(0)
            
            elif i > self.water_level:
               
                filtered_layers = self.Layer_Properties
                filtered_layers = filtered_layers[filtered_layers['non_soil'] != 'y']
                filtered_layers.loc[:, 'mlv_top_elev'] = filtered_layers['mlv_top_elev'].apply(pd.to_numeric)
                filtered_layers.loc[:, 'mlv_bottom_elev'] = filtered_layers['mlv_bottom_elev'].apply(pd.to_numeric)

                matching_row = filtered_layers[(filtered_layers['mlv_bottom_elev'] <= i) & 
                                                (i < filtered_layers ['mlv_top_elev'])]
                 
                layer_bottom = pd.to_numeric(matching_row.iloc[0]['mlv_bottom_elev'])
                layer_top = pd.to_numeric(matching_row.iloc[0]['mlv_top_elev'])
                unit_weight = pd.to_numeric(matching_row.iloc[0]['mlv_uw'])

                istress = self.eff_stress[loop_count-1] + unit_weight
                self.eff_stress.append(istress)

            elif i < self.water_level:
                
                filtered_layers = self.Layer_Properties
                filtered_layers = filtered_layers[filtered_layers['non_soil'] != 'y']
                filtered_layers.loc[:, 'mlv_top_elev'] = filtered_layers['mlv_top_elev'].apply(pd.to_numeric)
                filtered_layers.loc[:, 'mlv_bottom_elev'] = filtered_layers['mlv_bottom_elev'].apply(pd.to_numeric)

                matching_row = filtered_layers[(filtered_layers['mlv_bottom_elev'] <= i) & 
                                                (i < filtered_layers ['mlv_top_elev'])]
                 
                layer_bottom = pd.to_numeric(matching_row.iloc[0]['mlv_bottom_elev'])
                layer_top = pd.to_numeric(matching_row.iloc[0]['mlv_top_elev'])
                unit_weight = pd.to_numeric(matching_row.iloc[0]['mlv_uw']) - 62.4

                istress = self.eff_stress[loop_count-1] + unit_weight
                self.eff_stress.append(istress)

            loop_count = loop_count + 1
            loop_list.append(loop_count)
        return self.eff_stress

In [123]:
test_class = pile_design()

In [124]:
test_class.Pile_Effective_Stress()

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 42.6,
 85.2,
 127.80000000000001,
 170.4,
 213.0,
 255.6,
 298.2,
 340.8,
 383.40000000000003,
 426.00000000000006,
 468.6000000000001,
 511.2000000000001,
 553.8000000000001,
 596.4000000000001,
 639.0000000000001,
 681.6000000000001,
 724.2000000000002,
 766.8000000000002,
 809.4000000000002,
 852.0000000000002,
 894.6000000000003,
 937.2000000000003,
 979.8000000000003,
 1022.4000000000003,
 1065.0000000000002,
 1107.6000000000001,
 1145.2,
 1182.8,
 1220.3999999999999,
 1257.9999999999998,
 1295.5999999999997,
 1333.1999999999996,
 1370.7999999999995,
 1408.3999999999994,
 1445.9999999999993,
 1483.5999999999992,
 1521.1999999999991,
 1558.799999999999,
 1596.399999999999,
 1633.9999999999989,
 1671.5999999999988,
 1714.1999999999987,
 1756.7999999999986,
 1799.3999999999985,
 1841.9999999999984,
 1884.5999999999983,
 1927.1999999999982,
 1969.7999999999981,
 2012.399999999998,
 2054.999999999998,
 2097.599999999998,
 2140.199999999998,
 2182.7999999999